In [16]:
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line


In [17]:
# auto reload
%load_ext autoreload
%autoreload 2

import ephem
import geoplot
import imageio
import utm
from tqdm.auto import tqdm
from shapely.geometry import Point
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime, timedelta

from eolearn.core import FeatureType, LinearWorkflow, EOExecutor, EOTask, SaveTask, LoadTask, OverwritePermission, MergeFeatureTask, ExtractBandsTask
from eolearn.mask import AddMultiCloudMaskTask, AddValidDataMaskTask
from eolearn.io import SentinelHubInputTask
from eolearn.coregistration import ThunderRegistration
from eolearn.features import SimpleFilterTask, NormalizedDifferenceIndexTask
from sentinelhub import DataSource, BBox, CRS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Task definitions

In [18]:
# define EOTasks and utilities for below

class ValidCoverage(EOTask):
    def __init__(self, feature_in, feature_out):
        self.feature_in = feature_in
        self.feature_out = feature_out
        
    def execute(self, eopatch):
        mask = eopatch[self.feature_in]
        coverage = np.count_nonzero(mask == 1, axis=(1,2))/np.prod(mask.shape[1:])
        eopatch[self.feature_out] = coverage
        return eopatch
    
class SentinelHubValidData:
    def __init__(self, data_mask, clm_mask):
        self.data_mask = data_mask
        self.clm_mask = clm_mask
        
    def __call__(self, eopatch):
        return np.logical_and(eopatch.mask[self.data_mask].astype(np.bool),
                              np.logical_not(eopatch.mask[self.clm_mask].astype(np.bool)))

In [7]:
# initialize EOTasks

# download all L1C bands for cloud masking
download_task_l1c = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, 'BANDS-S2-L1C'),
    resolution=10,
    maxcc=1.0,
    time_difference=timedelta(days=2),
    data_source=DataSource.SENTINEL2_L1C,
    max_threads=10,
    additional_data=[
        (FeatureType.DATA, 'sunAzimuthAngles'),
        (FeatureType.DATA, 'sunZenithAngles')
    ]
)

# download RBG L2A bands for plotting
download_task_l2a = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, 'RGB'),
    bands = ['B04', 'B03', 'B02'],
    resolution=10,
    maxcc=1.0,
    time_difference=timedelta(days=2),
    data_source=DataSource.SENTINEL2_L2A,
    max_threads=10,
    additional_data=[(FeatureType.MASK, 'dataMask')]
)

# task for calculating clouds masks
add_clm_task = AddMultiCloudMaskTask(
    processing_resolution=160,
    mask_feature='CLM',
    is_data_feature = 'dataMask',
    average_over=16,
    dilation_size=8
)

# task for creating a valid data mask
valid_mask_task = AddValidDataMaskTask(SentinelHubValidData(data_mask='dataMask', clm_mask='CLM'), 'VALID_DATA')
        

# task for calculating the valid coverage
add_cov_task = ValidCoverage(feature_in=(FeatureType.MASK, 'VALID_DATA'), 
                             feature_out=(FeatureType.SCALAR, 'VALID_COVERAGE'))


# task for coregistrating the time frames
coreg_task = ThunderRegistration((FeatureType.DATA, 'RGB'), valid_mask_feature = (FeatureType.MASK, 'VALID_DATA'), 
                                 channel=0)


# tasks for loading and saving
load_task = LoadTask('./eopatches', lazy_loading=True)
save_task = SaveTask('./eopatches', overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

# Download

# Load

In [9]:
# load the previously created eopatches

workflow = LinearWorkflow(
    load_task
)

execution_args = []
for idx, bbox in enumerate(bbox_list):
    execution_args.append({
        load_task: {'eopatch_folder': f'eopatch_{idx}'}
    })
    
eopatches = []
for args in execution_args:
    eopatches.append(workflow.execute(args).eopatch())

# Create animation

In [10]:
# create RGB animations 

factors = [2.0, 2.5, 2.5, 2.75]

def plot_image(idx, f):
    fig = plt.figure(figsize=(10,10))
    plt.imshow(np.clip(eop.data['RGB'][idx]*f,0,1))
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.savefig(f'graphs/true_color_{name}_{idx}.png', dpi=50, bbox_inches='tight')
    plt.close()

for idx, f in tqdm(enumerate(factors), total=len(factors)):
    eop = eopatches[idx]
    name = f'loc{idx}'
    
    def plot(idx):
        plot_image(idx,f)

    os.system('rm -rf graphs/*')
    with ProcessPoolExecutor(max_workers=8) as executor:
        _ = list(tqdm(executor.map(plot, range(len(eop.timestamp))), total=len(eop.timestamp), leave=False))

    n_valid = np.count_nonzero(eop.scalar['VALID_COVERAGE'] == 1)
    with imageio.get_writer(f'figs/true_color_{name}.gif', mode='I', duration=4.0/n_valid) as writer:
        for i in range(len(eop.timestamp)):
            if eop.scalar['VALID_COVERAGE'][i] == 1:
                image = imageio.imread(f'graphs/true_color_{name}_{i}.png')
                writer.append_data(image)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Create graph animation

In [14]:
# define function for plotting time frame

def plot_graph(idx, f, name):
    x = np.mean(eop.data['sunAzimuthAngles'][...,0], axis=(1,2))
    y = np.mean(eop.data['sunZenithAngles'][...,0], axis=(1,2))
    valid_mask = eop.scalar['VALID_COVERAGE'].squeeze() == 1.0
    ids = np.array(range(len(valid_mask)))
    dates = np.array([ts.date().isoformat()[:-3] for ts in eop.timestamp])
    transition_dates = [datetime(2019,3,20), datetime(2019,6,21),datetime(2019,9,23), datetime(2019,12,22)]
    transition_doys = [(x-datetime(2019,1,1)).days for x in transition_dates]    
    try:
        last_valid = ids[:idx+1][valid_mask[:idx+1]][-1]
    except:
        last_valid = None
    
    fig, axs = plt.subplots(2,2,figsize=(10,10))
    plot_ids = np.array(range(len(valid_mask[:idx])))[valid_mask[:idx]].astype(int)

    ax1 = axs[0,0]
    ax1.plot(x,y)
    ax1.plot(x[:idx+1],y[:idx+1],'r')
    ax1.plot(x[plot_ids],y[plot_ids],'kx')
    ax1.plot(x[idx], y[idx], 'r', marker='o')
    ax1.set_xlabel('Zenith [°]')
    ax1.set_ylabel('Azimuth [°]')

    def idx_to_days(x):
        return x*365/len(dates)

    def days_to_idx(x):
        return x*len(dates)/365
    
    ax2 = axs[0,1]
    ax2.plot(y)
    ax2.plot(range(len(y[:idx+1])), y[:idx+1],'r')
    ax2.plot(np.array(range(len(y)))[plot_ids], y[plot_ids],'kx')
    ax2.plot(idx, y[idx], 'r', marker='o')
    ax2.set_xlabel('Time')
    ax2.set_xticks([days_to_idx(doy) for doy in transition_doys])
    ax2.set_xticklabels([dt.strftime(format='%Y-%m') for dt in transition_dates], ha='right')
    ax2.set_yticks([])
    
    ax3 = axs[1,0]
    ax3.plot(x, range(len(x)))
    ax3.plot(x[:idx+1], range(len(x[:idx+1])), 'r')
    ax3.plot(x[plot_ids], np.array(range(len(x)))[plot_ids], 'kx')
    ax3.plot(x[idx], idx, 'r', marker='o')
    ax3.set_xticks([])
    ax3.set_yticks([])
    
    sax3 = ax3.secondary_yaxis('right', functions=(idx_to_days, days_to_idx))
    sax3.set_ylabel('Time')
    sax3.set_yticks(transition_doys)
    sax3.set_yticklabels([dt.strftime(format='%Y-%m') for dt in transition_dates], rotation=90, va='top')

    ax1.axvline(x=x[idx],ymin=-1.2,ymax=1,c="gray",linewidth=1, linestyle='dashed',zorder=0, clip_on=False)
    ax3.axvline(x=x[idx],ymin=0,ymax=1,c="gray",linewidth=1, linestyle='dashed',zorder=0, clip_on=False)
    ax1.axhline(y=y[idx],xmin=0,xmax=1.2,c="gray",linewidth=1, linestyle='dashed',zorder=0, clip_on=False)
    ax2.axhline(y=y[idx],xmin=0,xmax=1,c="gray",linewidth=1, linestyle='dashed',zorder=0, clip_on=False)
    
    ax4 = axs[1,1]
    
    if last_valid is not None:
        ax4.imshow(np.clip(eop.data['RGB'][last_valid]*f,0,1))
    else:
        pass
    
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.axis('off')  
        
    # comment out below to plot
    plt.savefig(f'graphs/graph_{name}_{idx}.png', dpi=100, bbox_inches='tight')
    plt.close()

In [15]:
# create RGB and solar angle animation
idx = 0
eop = eopatches[idx]
name = f'loc{idx}'
f = factors[idx]

def plot(idx):
    plot_graph(idx, f, name)

os.system('rm -rf graphs/*')
with ProcessPoolExecutor(max_workers=8) as executor:
    _ = list(tqdm(executor.map(plot, range(len(eop.timestamp))), total=len(eop.timestamp)))

with imageio.get_writer(f'figs/graph_{name}.gif', mode='I', duration=4.0/len(eop.timestamp)) as writer:
    for i in range(len(eop.timestamp)):
        image = imageio.imread(f'graphs/graph_{name}_{i}.png')
        writer.append_data(image)